## **Configuration**

In [ ]:
import numpy as np
import cv2
import os
from google.colab import drive
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import datasets, layers, models, losses
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
import pandas as pd
from tabulate import tabulate

In [ ]:
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **Loading dataset**

In [ ]:
#connection to my drive
path ="/content/drive/My Drive/ExtendedYaleB"
#images list
images = []
#labels list
labels = []

#loop over images folders
for i in range(1, 39):
    if i < 10:
        subdirPath = os.path.join(path, f'yaleB0{i}')
    #folder 14 doesnot exist
    elif i == 14:
        subdirPath = os.path.join(path, f'yaleB{i+1}')
        i = i +1
    else:
        subdirPath = os.path.join(path, f'yaleB{i}')
    #loop over each file images
    for filename in os.listdir(subdirPath):
        #some folders doesnot contain images only so this if conditon to loop over images only
        if filename.endswith('.pgm'):
            imgPath = os.path.join(subdirPath, filename)
            img = cv2.imread(imgPath, cv2.IMREAD_GRAYSCALE)
            # Resize the image to 64x64
            img = cv2.resize(img, (64, 64))
            images.append(img)
            # Extract the label from the filename and append it to the labels list
            label = int(filename.split("_")[0][-2:]) - 1
            labels.append(label)


In [ ]:
#list conversion into array
listToArray = np.array(images)


## **Splitting**

In [ ]:
#split the dataset into training 80%, validation 10%, and testing 10% sets

x_train, x_test, y_train, y_test = train_test_split(listToArray,labels,test_size=0.1)
x_train, x_val,y_train, y_val = train_test_split(x_train,y_train,test_size=0.1)

In [ ]:
#changing the range to be between [0,1]
x_train = x_train / 255.0
x_val = x_val / 255.0
x_test = x_test/ 255.0
# converting the values to be numpy array
y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)


In [ ]:
# reshaping the dataset
x_train = x_train.reshape((-1, 64, 64, 1))
x_val = x_val.reshape((-1, 64, 64, 1))
x_test = x_test.reshape((-1, 64, 64, 1))

## **MLP Buliding**

In [ ]:
# Build a Multilayer Perceptron model
mlpModel = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(64, 64)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(38, activation='softmax')
])

In [ ]:
mlpModel.summary()

Model: "sequential_43"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_30 (Flatten)        (None, 4096)              0         
                                                                 
 dense_79 (Dense)            (None, 128)               524416    
                                                                 
 dense_80 (Dense)            (None, 64)                8256      
                                                                 
 dense_81 (Dense)            (None, 38)                2470      
                                                                 
Total params: 535,142
Trainable params: 535,142
Non-trainable params: 0
_________________________________________________________________


## **CNN Building**

In [ ]:
# Build a Convolutional Neural Network (CNN) model
cnnModel = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(38, activation='softmax')
])

In [ ]:
cnnModel.summary()

Model: "sequential_44"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_27 (Conv2D)          (None, 62, 62, 32)        320       
                                                                 
 max_pooling2d_27 (MaxPoolin  (None, 31, 31, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_28 (Conv2D)          (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_28 (MaxPoolin  (None, 14, 14, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_29 (Conv2D)          (None, 12, 12, 128)       73856     
                                                                 
 max_pooling2d_29 (MaxPoolin  (None, 6, 6, 128)      

## **Set Loss Function & Optimizer**

In [ ]:
# Sparse Categorical loss function is used because w have a wide range of labels
mlpModel.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Sparse Categorical loss function is used because w have a wide range of labels
cnnModel.compile(optimizer="adam", loss=losses.SparseCategoricalCrossentropy(), metrics=["accuracy"])

## **Train model**

In [ ]:
# Model training with the defult batch size (32) and 30 ephocs
history = mlpModel.fit(x_train, y_train, epochs=30 , validation_data=(x_val, y_val))

Epoch 1/30
62/62 [==============================] - 2s 13ms/step - loss: 3.6004 - accuracy: 0.0464 - val_loss: 3.4536 - val_accuracy: 0.0769
Epoch 2/30
62/62 [==============================] - 1s 10ms/step - loss: 3.1266 - accuracy: 0.1991 - val_loss: 2.8390 - val_accuracy: 0.2443
Epoch 3/30
62/62 [==============================] - 1s 10ms/step - loss: 2.3175 - accuracy: 0.4375 - val_loss: 1.8923 - val_accuracy: 0.6154
Epoch 4/30
62/62 [==============================] - 1s 10ms/step - loss: 1.5928 - accuracy: 0.6517 - val_loss: 1.3420 - val_accuracy: 0.7421
Epoch 5/30
62/62 [==============================] - 1s 10ms/step - loss: 1.1496 - accuracy: 0.7510 - val_loss: 0.9421 - val_accuracy: 0.8462
Epoch 6/30
62/62 [==============================] - 1s 9ms/step - loss: 0.8196 - accuracy: 0.8422 - val_loss: 0.7588 - val_accuracy: 0.8643
Epoch 7/30
62/62 [==============================] - 1s 10ms/step - loss: 0.6460 - accuracy: 0.8841 - val_loss: 0.6767 - val_accuracy: 0.8552
Epoch 8/30
62/

In [ ]:
# Model training with the defult batch size (32) and 30 ephocs
history = cnnModel.fit(x_train, y_train, epochs=30, validation_data=(x_test, y_test))


Epoch 1/30
62/62 [==============================] - 19s 281ms/step - loss: 3.6266 - accuracy: 0.0449 - val_loss: 3.5919 - val_accuracy: 0.0732
Epoch 2/30
62/62 [==============================] - 19s 308ms/step - loss: 3.5333 - accuracy: 0.0685 - val_loss: 3.3942 - val_accuracy: 0.1423
Epoch 3/30
62/62 [==============================] - 18s 290ms/step - loss: 2.8688 - accuracy: 0.2172 - val_loss: 1.8891 - val_accuracy: 0.5325
Epoch 4/30
62/62 [==============================] - 17s 276ms/step - loss: 1.6002 - accuracy: 0.5509 - val_loss: 0.9792 - val_accuracy: 0.7439
Epoch 5/30
62/62 [==============================] - 17s 281ms/step - loss: 1.0134 - accuracy: 0.6976 - val_loss: 0.6434 - val_accuracy: 0.8496
Epoch 6/30
62/62 [==============================] - 17s 280ms/step - loss: 0.5956 - accuracy: 0.8196 - val_loss: 0.4041 - val_accuracy: 0.8984
Epoch 7/30
62/62 [==============================] - 17s 279ms/step - loss: 0.3876 - accuracy: 0.8846 - val_loss: 0.3000 - val_accuracy: 0.9350

## **Evaluate model**

In [ ]:
mlpLoss, mlpAcc = mlpModel.evaluate(x_test, y_test)


8/8 [==============================] - 0s 4ms/step - loss: 0.9942 - accuracy: 0.7358


In [ ]:
cnnLoss, cnnAcc = cnnModel.evaluate(x_test, y_test)

8/8 [==============================] - 0s 59ms/step - loss: 0.1760 - accuracy: 0.9756


# **Bonus**

## **Data augmentation**

In [ ]:
dataAugmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.03),
    #layers.RandomTranslation(height_factor = (-0.2,1), width_factor = (-0.2,1), fill_mode ="reflect"),
    layers.RandomContrast(factor=(0.1,0.2),seed=None),
])

## **MLP With Data augmentation**

In [ ]:
# Build a Multilayer Perceptron Model With Data Augmentation
mlpModelAug = tf.keras.models.Sequential([
    dataAugmentation,
    tf.keras.layers.Flatten(input_shape=(64, 64)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(38, activation='softmax')
])

In [ ]:
#Training/Testing the model
mlpModelAug.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the MLP model on the training set (with data augmentation), default batch size and 30 epochs
mlpHistoryAug = mlpModelAug.fit(x_train, y_train, epochs=30 , validation_data=(x_val, y_val))

# Evaluate the MLP model on the testing set (with data augmentation)
mlpLossAug, mlpAccAug = mlpModelAug.evaluate(x_test, y_test)

Epoch 1/30
62/62 [==============================] - 3s 20ms/step - loss: 3.6667 - accuracy: 0.0413 - val_loss: 3.5652 - val_accuracy: 0.0588
Epoch 2/30
62/62 [==============================] - 1s 17ms/step - loss: 3.4618 - accuracy: 0.0897 - val_loss: 3.2644 - val_accuracy: 0.1629
Epoch 3/30
62/62 [==============================] - 1s 17ms/step - loss: 3.1436 - accuracy: 0.1578 - val_loss: 2.8067 - val_accuracy: 0.2398
Epoch 4/30
62/62 [==============================] - 1s 16ms/step - loss: 2.8132 - accuracy: 0.2298 - val_loss: 2.5089 - val_accuracy: 0.2941
Epoch 5/30
62/62 [==============================] - 1s 16ms/step - loss: 2.4861 - accuracy: 0.3100 - val_loss: 2.1586 - val_accuracy: 0.3982
Epoch 6/30
62/62 [==============================] - 1s 16ms/step - loss: 2.1213 - accuracy: 0.4229 - val_loss: 1.7096 - val_accuracy: 0.4842
Epoch 7/30
62/62 [==============================] - 1s 17ms/step - loss: 1.8775 - accuracy: 0.4909 - val_loss: 1.5555 - val_accuracy: 0.5792
Epoch 8/30
62

## **CNN with data augmentation**

In [ ]:
cnnModelAug = Sequential([
    dataAugmentation,
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(38, activation='softmax')
])


In [ ]:
#Training/Testing the model
cnnModelAug.compile(optimizer="adam", loss=losses.SparseCategoricalCrossentropy(), metrics=["accuracy"])
# Train the CNN model on the training set (with data augmentation)
cnnHistoryAug = cnnModelAug.fit(x_train, y_train,epochs=30, validation_data=(x_val, y_val))

# Evaluate the CNN model on the testing set (with data augmentation)
cnnLossAug, cnnAccAug = cnnModelAug.evaluate(x_test, y_test)

Epoch 1/30
62/62 [==============================] - 19s 283ms/step - loss: 3.6231 - accuracy: 0.0484 - val_loss: 3.6116 - val_accuracy: 0.0407
Epoch 2/30
62/62 [==============================] - 22s 349ms/step - loss: 3.5659 - accuracy: 0.0529 - val_loss: 3.4475 - val_accuracy: 0.0543
Epoch 3/30
62/62 [==============================] - 18s 296ms/step - loss: 3.1457 - accuracy: 0.1578 - val_loss: 2.5454 - val_accuracy: 0.3529
Epoch 4/30
62/62 [==============================] - 17s 283ms/step - loss: 2.3702 - accuracy: 0.3523 - val_loss: 1.6670 - val_accuracy: 0.5385
Epoch 5/30
62/62 [==============================] - 18s 283ms/step - loss: 1.7116 - accuracy: 0.4985 - val_loss: 1.0526 - val_accuracy: 0.7014
Epoch 6/30
62/62 [==============================] - 17s 282ms/step - loss: 1.3135 - accuracy: 0.6028 - val_loss: 0.7080 - val_accuracy: 0.8416
Epoch 7/30
62/62 [==============================] - 18s 290ms/step - loss: 1.0617 - accuracy: 0.6694 - val_loss: 0.4446 - val_accuracy: 0.9095

## **Displaying the data**

In [ ]:
# Define the table data as a list of lists
table_data = [
    ['MLP without augmentation', mlpLoss, mlpAcc , np.nan, np.nan],
    ['MLP with augmentation', np.nan, np.nan, mlpLossAug, mlpAccAug],
    ['CNN without augmentation', cnnLoss, cnnAcc , np.nan, np.nan],
    ['CNN with augmentation', np.nan, np.nan, cnnLossAug, cnnAccAug]
]

# Define the table headers
headers = ['Model', 'Loss (without augmentation)', 'Accuracy (without augmentation)', 'Loss (with augmentation)', 'Accuracy (with augmentation)']

# Print the table using the tabulate package
print(tabulate(table_data, headers=headers))



Model                       Loss (without augmentation)    Accuracy (without augmentation)    Loss (with augmentation)    Accuracy (with augmentation)
------------------------  -----------------------------  ---------------------------------  --------------------------  ------------------------------
MLP without augmentation                       0.994154                           0.735772                  nan                             nan
MLP with augmentation                        nan                                nan                           0.497098                        0.878049
CNN without augmentation                       0.176026                           0.97561                   nan                             nan
CNN with augmentation                        nan                                nan                           0.130791                        0.97561
